In [47]:
import xml.sax

class GroundTruthHandler(xml.sax.ContentHandler):
    def __init__(self, urls):
        xml.sax.ContentHandler.__init__(self)
        self.urls = urls

    def startElement(self, name, attrs):
        if name == "article":
            articleId = attrs.getValue("id")
            hyperpartisan = attrs.getValue("hyperpartisan")
            url = attrs.getValue("url")
            # parse url
            url = '/'.join(url.split('/')[:3])
            if url in self.urls.keys():
                self.urls[url].append(hyperpartisan)
            else:
                self.urls[url] = [hyperpartisan]

In [48]:
urls = {}
yTst = "data/ground-truth-training-byarticle.xml"
with open(yTst) as f:
    xml.sax.parse(f, GroundTruthHandler(urls))

In [49]:
import collections
def count(label):
    return collections.Counter(j)

In [50]:
dct = {}
for i, j in urls.items():
    if len(j) > 3:
        dct[i] = 100* count(j)['true'] /(count(j)['true'] + count(j)['false'])

In [51]:
url_dict = {
'left-center': ['http://www.cbsnews.com', 'http://heavy.com', 'http://www.nytimes.com'],
'right-center': ['http://nypost.com'],
'center': ['http://www.snopes.com','https://www.realclearpolitics.com','https://www.circa.com'],
'right':['https://pjmedia.com', 'http://www.express.co.uk','http://opslens.com','http://insider.foxnews.com', 'http://www.foxnews.com', 'http://www.washingtonexaminer.com'],
'left':["https://www.salon.com", 'http://www.gq.com'],
'extreme':['http://www.thegatewaypundit.com', 'http://www.trueactivist.com', "http://www.dcclothesline.com", 'https://bearingarms.com']}

In [52]:
for bias in url_dict.keys():
    print("="*20 + bias + "="*20)
    for url in url_dict[bias]:
        print(url, dct[url])

====================left-center====================
http://www.cbsnews.com 11.11111111111111
http://heavy.com 7.6923076923076925
http://www.nytimes.com 30.0
====================right-center====================
http://nypost.com 66.66666666666667
====================center====================
http://www.snopes.com 8.333333333333334
https://www.realclearpolitics.com 92.85714285714286
https://www.circa.com 12.5
====================right====================
https://pjmedia.com 100.0
http://www.express.co.uk 36.8421052631579
http://opslens.com 100.0
http://insider.foxnews.com 27.272727272727273
http://www.foxnews.com 50.0
http://www.washingtonexaminer.com 57.142857142857146
====================left====================
https://www.salon.com 100.0
http://www.gq.com 60.0
====================extreme====================
http://www.thegatewaypundit.com 94.44444444444444
http://www.trueactivist.com 62.5
http://www.dcclothesline.com 85.71428571428571
https://bearingarms.com 66.66666666666667
